**2. Working with ensemble data**

To do this you will need to download "HadSST.4.0.0.0_ensemble.zip" from https://www.metoffice.gov.uk/hadobs/hadsst4/data/download.html it is a massive folder of 200 ensemble members! A useful tool for looking at netcdf files: Panoply (Nasa) https://www.giss.nasa.gov/tools/panoply/ 

Model ensembles are very common. They represent a group (anywhere between 2 and 100+) of *model runs* using similar initial conditions. The initial conditions are varied slightly for each run to mimic randomness within the earth system. 

1. Initialise any arrays

2. Create a for loop which loops through the files in the ensemble foler and loads each sst file and the variables

3. Within the for loop calculate the baseline, current climates and anomalies for each of the files
uses formulas from last week 

4. For each type of climate: baselines, currents and anomalies, concatenate the arrays so you end up with three arrays of shape (200,36,72) (as you have 200 files, you will have 200 values for each grid point) 
data2 = pd.concat([year, anomaly], axis=1) # learned to concatenate two series into a df

5. outside of the for loop calculate the anomaly mean (for each grid point)



6. Now loop through all of the grid points and compare the distributions of the baseline compared to the currents - are they significantly different (i.e. is the global sst now significantly different from what it was during the baseline?)


You should output:

* an anomaly array of shape (36,72) which represents current - baseline SST
* a boolean array of shape (36,72) which represents whether there is a statistical significant difference between baseline and current
So this is not straightforward, if you're finding it impossible then just use the answers for this bit and try to understand what each line is doing.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from netCDF4 import Dataset,num2date
from datetime import date, timedelta

**1. Initialise any arrays** 

In [2]:
np.zeros((36,72))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**2. Create a for loop which loops through the files in the ensemble foler and loads each sst file and the variables**

In [3]:
import os
directory = '/Users/bendixon/Documents/HadSST/'
for filename in os.listdir(directory): 
    print(filename)
    # sst_file = Dataset(filename)
    # variables = sst_file.variables

HadSST.4.0.0.0_ensemble_member_67.nc
HadSST.4.0.0.0_ensemble_member_36.nc
HadSST.4.0.0.0_ensemble_member_94.nc
HadSST.4.0.0.0_ensemble_member_53.nc
HadSST.4.0.0.0_ensemble_member_43.nc
HadSST.4.0.0.0_ensemble_member_12.nc
HadSST.4.0.0.0_ensemble_member_84.nc
HadSST.4.0.0.0_ensemble_member_26.nc
HadSST.4.0.0.0_ensemble_member_77.nc
HadSST.4.0.0.0_ensemble_member_57.nc
HadSST.4.0.0.0_ensemble_member_198.nc
HadSST.4.0.0.0_ensemble_member_32.nc
HadSST.4.0.0.0_ensemble_member_63.nc
HadSST.4.0.0.0_ensemble_member_90.nc
HadSST.4.0.0.0_ensemble_member_4.nc
HadSST.4.0.0.0_ensemble_member_80.nc
HadSST.4.0.0.0_ensemble_member_73.nc
HadSST.4.0.0.0_ensemble_member_22.nc
HadSST.4.0.0.0_ensemble_member_16.nc
HadSST.4.0.0.0_ensemble_member_188.nc
HadSST.4.0.0.0_ensemble_member_47.nc
HadSST.4.0.0.0_ensemble_member_56.nc
HadSST.4.0.0.0_ensemble_member_199.nc
HadSST.4.0.0.0_ensemble_member_33.nc
HadSST.4.0.0.0_ensemble_member_62.nc
HadSST.4.0.0.0_ensemble_member_91.nc
HadSST.4.0.0.0_ensemble_member_5.nc


In [33]:
for filename in os.listdir(directory):
    swta_file = Dataset(filename)
    variables = swta_file.variables
    swta = swta_file.variables['tos'][:]
    print(swta[0])

FileNotFoundError: [Errno 2] No such file or directory: b'HadSST.4.0.0.0_ensemble_member_67.nc'

In [54]:
# I think the action here is to use a loop to fill in an array
# could try this out by making a loop to fill another array
a = np.zeros((4,20))
for i in range(1,20):
    a[0,i] == 5

**3. Within the for loop calculate the baseline, current climates and anomalies for each of the files**

In [4]:
# Importing the sst file
fn = '/Users/bendixon/Documents/HadSST/HadSST.4.0.0.0_ensemble_member_1.nc'
swta_file = Dataset(fn)
variables = swta_file.variables
swta = swta_file.variables['tos'][:]
time = swta_file.variables['time'][:]
latitude = swta_file.variables['latitude'][:]
longitude = swta_file.variables['longitude'][:]
swta = np.ma.masked_values(swta, 9.969209968386869e+36)

In [31]:
time_units = swta_file.variables['time'].units
calendar = swta_file.variables['time'].calendar
time_convert = num2date(time, time_units, calendar)

In [6]:
# Find the position in the time array corresponding to 1950, 1, 1 
# Looking for a more automated way, e.g. 
# DatetimeGregorian(1850, 2, 15, 0, 0, 0, 0)
# np.where(time_convert == DateTime?)
# date(1850,1,16).toordinal() - this gives the day number, but we want the number of the row in the array

In [7]:
print(time_convert[1200])
print(time_convert[1560]) # not sure why some months there are measurements at 00:00 or 12:00
print(time_convert[1800])
print(time_convert[2027]) # this is only up to 2018, but that's the size of the array

1980-01-16 12:00:00
1950-01-16 12:00:00
2000-01-16 12:00:00
2018-12-16 12:00:00


In [29]:
i_1950 = 1200; i_1980 = 1569; i_2000 = 1800; i_2018 = 2027

baseline = swta[i_1950:i_1980,:,:]
baseline = np.mean(baseline,axis=0) 

current = swta[i_2000:i_2018,:,:]
current = np.mean(current,axis=0)

# current - baseline
anomaly = current - baseline

**Workings/ideas**

Overall process:
1. Create an array
2. Loop through each file and find the 

In [ ]:


# initialise empty arrays

# open data in for loop
for i in range(1,200):

    # calculate baseline and current global mean sst
    
    # you can't np.concatenate to an empty array so create an if statement to deal with the first instance    


# calculate ensemble mean sst anomaly for each grid point

# loop through each grid point and assess whether the current values are significantly different from the baseline
for i in range(36):
    for j in range(72):
        # ...     

anomaly = 
significance =